# Understanding the MSI file
The Matrix Signal Information (MSI) file describes the information that is shown on the matrix signs above highways. This information can be imported in a csv file, where each line describes a change in MSI. We want to process this information, until we can deduce:
- For any time and any location on the highway (hectometer marker, inside the range), the available lanes and maximum speeds.

We will find this by first looking in the excel file "backwards over the highway", looking for the closest matrix sign. Then, we will search backwards in time for the latest update.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [4]:
# Define paths
datafolder = os.path.join(os.path.dirname(os.getcwd()), "data", "short_highway")
datafolder_msi = os.path.join(datafolder, "msi-export")
file1_path = os.path.join(datafolder_msi, "msi-export.csv")

In [7]:
# import file into pd.df
df = pd.read_csv(file1_path, low_memory=False)

In [8]:
print(df)

     Datum en tijd beeldstandwijziging Datum en tijd bericht verzonden   
0                  2023-10-15 18:50:21             2023-10-15 18:50:24  \
1                  2023-10-15 18:50:21             2023-10-15 18:50:24   
2                  2023-10-15 18:50:21             2023-10-15 18:50:24   
3                  2023-10-15 18:50:21             2023-10-15 18:50:24   
4                  2023-10-15 18:50:21             2023-10-15 18:50:24   
...                                ...                             ...   
4149               2023-10-16 23:48:45             2023-10-16 23:48:46   
4150               2023-10-16 23:48:45             2023-10-16 23:48:46   
4151               2023-10-16 23:48:45             2023-10-16 23:48:46   
4152               2023-10-16 23:48:46             2023-10-16 23:48:48   
4153               2023-10-16 23:48:50             2023-10-16 23:48:50   

     Datum en tijd bericht ontvangen   
0                2023-10-15 18:50:24  \
1                2023-10-15 18: